In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas
import numpy as np
import tensorflow as tf

In [21]:
# Read in training data, popping off what we want to predict
train_data = pandas.read_csv('train.csv')
test_data = pandas.read_csv('test.csv')

test_pIds = test_data.get('PassengerId')

y_data = train_data.pop('Survived')
y_train = np.zeros((len(y_data), 2)) # two categories, survived or not survived
y_train[np.arange(len(y_data)), y_data] = 1

# === Clean Data ===
train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
medAge = train_data.Age.median()
medFare = train_data.Fare.median()
train_data['Age'].fillna(medAge, inplace=True)
train_data['Fare'].fillna(medFare, inplace=True)
train_data['Sex'].replace('male', 0, inplace=True)
train_data['Sex'].replace('female', 1, inplace=True)
train_data['Embarked'].replace('S', 0, inplace=True)
train_data['Embarked'].replace('C', 1, inplace=True)
train_data['Embarked'].replace('Q', 2, inplace=True)
X_train = train_data.values

# Remove anywhere a non number exists (Embarked)
y_train = y_train[~np.isnan(X_train).any(axis=1)]
X_train = X_train[~np.isnan(X_train).any(axis=1)]

test_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_data['Age'].fillna(medAge, inplace=True)
test_data['Fare'].fillna(medFare, inplace=True)
test_data['Sex'].replace('male', 0, inplace=True)
test_data['Sex'].replace('female', 1, inplace=True)
test_data['Embarked'].replace('S', 0, inplace=True)
test_data['Embarked'].replace('C', 1, inplace=True)
test_data['Embarked'].replace('Q', 1, inplace=True)
X_test = test_data.values

X_test = X_test[~np.isnan(X_test).any(axis=1)]



In [4]:

num_inputs = len(X_train[0])
num_outputs = len(y_train[0]); # survived or not survived

# Input

x = tf.placeholder(tf.float32, (None, num_inputs), name='x')
y = tf.placeholder(tf.float32, (None, num_outputs), name='y')
ec = tf.argmax(y, 1, name='ec')

# Output

b = tf.Variable(tf.zeros(num_outputs), name='b')
W = tf.Variable(tf.truncated_normal([num_inputs, num_outputs]), name='W')
a = tf.add(tf.matmul(x, W), b, name='a')
z = tf.nn.softmax(a, name='z')
pc = tf.argmax(z, 1, name='pc')  # [p]redicted [c]lass label

# Loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=a)
loss = tf.reduce_mean(cross_entropy, name='loss')

# Solver
solver = tf.train.AdamOptimizer()
optimize = solver.minimize(loss)

# Accuracy
correct = tf.equal(ec, pc, name='correct')  # for each sample, did we get it right? 
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='acc')

In [5]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
# Current Accuracy (before training)
print ("Accuracy:", sess.run(accuracy, {x:X_train, y:y_train}))
print ("Random chance is around 0.5 (50%)")

Accuracy: 0.799775
Random chance is around 0.5 (50%)


In [19]:
# Training cycle
training_epochs=10000
display_epoch=1000

# try:
#     saver.restore(sess, './models/logreg.cpt')
# except:
#     print "No model to restore"

for epoch in range(training_epochs):
    avg_loss = 0.
       
    _, loss_ = sess.run([optimize, loss],
                            {x: X_train, y: y_train})
        
    
    # Display logs per epoch step
    if (epoch+1) % display_epoch == 0:
        print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(loss_))

Epoch: 1000 cost= 0.441899657
Epoch: 2000 cost= 0.441924125
Epoch: 3000 cost= 0.441900223
Epoch: 4000 cost= 0.441899836
Epoch: 5000 cost= 0.441899776
Epoch: 6000 cost= 0.441899657
Epoch: 7000 cost= 0.441923201
Epoch: 8000 cost= 0.441902936
Epoch: 9000 cost= 0.441899687
Epoch: 10000 cost= 0.441899776


In [22]:
pred_test = sess.run([pc], {x:X_test})[0]

ret = pandas.DataFrame({'PassengerId':test_pIds, 'Survived':pred_test})
ret.to_csv('kaggle_submission_tensorflow_1.csv', index=False)